In [ ]:
import pandas as pd
from selenium.webdriver import Edge, EdgeOptions
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

# Edge driver path
edge_driver_path = 'C:/*******s/Spotify 2024/driver/msedgedriver.exe'



In [ ]:
# Setting up the url for spotify charts with the last thursday date
import datetime

today = datetime.date.today()

weekday = today.weekday()

days_to_last_thursday = (weekday - 3) % 7

last_thursday = today - datetime.timedelta(days=days_to_last_thursday)

last_thursday_str = last_thursday.strftime("%Y-%m-%d")

print("Last Thursday:", last_thursday_str)

url = f"https://charts.spotify.com/charts/view/regional-global-weekly/{last_thursday_str}"


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '******************'

client_secret = '*********************'

user = '*********'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
import pyodbc

server = 'juanlopezch.database.windows.net'
database = 'Spotify_charts'
username = '************'
password = '***********'

max_retries = 3
retry_delay = 60 

retry_count = 0
connected = False

while retry_count < max_retries and not connected:
    try:
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
        cursor = cnxn.cursor()
        connected = True
    except pyodbc.OperationalError as e:
        print(f"Connection Error: {e}")
        print(f"Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
        retry_count += 1

if connected:
    print("¡Successfuly Connected!")
else:
    print("Could not connect. Check server availability.")


In [ ]:
# Selenium script to download weekly charts
options = EdgeOptions()
service = Service(edge_driver_path)
driver = Edge(service=service, options=options)


driver.get('https://charts.spotify.com/home')

driver.maximize_window()

boton_login = driver.find_element("xpath", '//*[@id="__next"]/div/div/main/div[2]/div/header/div/div[2]/a/div[1]')
boton_login.click()

campo_usuario = driver.find_element("xpath", '//*[@id="login-username"]')
campo_usuario.send_keys("********")

campo_contrasena = driver.find_element("xpath", '//*[@id="login-password"]')
campo_contrasena.send_keys("*******")

boton_login = driver.find_element("xpath", '//*[@id="login-button"]/span[1]/span')
boton_login.click()

time.sleep(3)

driver.get(url)

time.sleep(3)

boton_download = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/div/main/div[2]/div[3]/div/div/a/button'))
)

y = boton_download.location['y']

driver.execute_script("window.scrollTo(0, {});".format(y - 200))  
boton_download.click()

time.sleep(3)

driver.quit()


In [ ]:
#Read Downloaded File
file_name = f'regional-global-weekly-{last_thursday_str}'

file_path = f'C:/Users/JUAN LOPEZ/Downloads/{file_name}.csv'

df = pd.read_csv(file_path)

df['end_of_week'] = last_thursday_str
df['end_of_week'] = pd.to_datetime(df['end_of_week'], format='%Y-%m-%d', errors = 'coerce')

df.info()

In [ ]:
#create UniqueID

df["UniqueID"] = df.apply(lambda x: x["uri"].split(":")[-1] + 
                                      str(x["rank"]) + 
                                      str(x["end_of_week"].year)[2:] + 
                                      str(x["end_of_week"].month).zfill(2) +
                                      str(x["end_of_week"].day).zfill(2), axis=1)


In [ ]:
# Remove ' and "

df['track_name'] = df['track_name'].str.replace(r"'", '')
df['artist_names'] = df['artist_names'].str.replace(r"'", '')
df['source'] = df['source'].str.replace(r"'", '')

In [ ]:
#Insert Dataframe into SQL Server:
for index, row in df.iterrows():
     cursor.execute("INSERT INTO dbo.WeeklyCharts (UniqueID,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,end_of_week) values(?,?,?,?,?,?,?,?,?,?,?)", 
                    row['UniqueID'], row['rank'], row['uri'], row['artist_names'], row['track_name'], row['source'], row['peak_rank'], row['previous_rank'], row['weeks_on_chart'], row['streams'], row['end_of_week'])
     cnxn.commit()


cursor.close()
print("Data added on dbo.WeeklyCharts")

In [ ]:
# Verify new songs
# Bring unique from DB

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

sql_query = "select uri from dbo.UniqueSongs"

unique_songs_sql = pd.read_sql(sql_query, cnxn)
cnxn.close()

In [ ]:
# Get unique uris from data downloaded and verify if is not yet created on SQL
uris_df_set = set(df['uri'].unique())

uris_sql_set = set(unique_songs_sql['uri'].unique())

uris_nuevas = uris_df_set - uris_sql_set

uris_nuevas_df = pd.DataFrame(list(uris_nuevas), columns=['uri'])

print(uris_nuevas_df.info())

In [ ]:
# Add new uris info on SQL
if len(uris_nuevas_df) > 0:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    
    for index, row in uris_nuevas_df.iterrows():
        try:

            track_id = row['uri']
            track_info = sp.track(track_id)

            imagen_grande = track_info['album']['images'][0]['url']
            popularidad = track_info['popularity']
            explicita = track_info['explicit']

            temp_df = pd.DataFrame({'uri': [track_id], 'Cover': [imagen_grande], 
                                    'popularity': [popularidad], 'explicit': [explicita]})

            cursor.execute("INSERT INTO dbo.UniqueSongs (uri,Cover,Popularity,IsExplicit) values(?,?,?,?)", 
                           row['uri'], imagen_grande, popularidad, explicita)
            cnxn.commit()
            
        except Exception as e:
            print(f"Error processing the uri {row['uri']}: {str(e)}")
    
    cursor.close()
    cnxn.close()

    print("Uris added succesfuly.")